In [1]:
! pip install llama-index-embeddings-ollama

In [3]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3.1:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print(pass_embedding)

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(query_embedding)

Generating embeddings: 100%|██████████| 2/2 [00:05<00:00,  2.85s/it]

[[-1.1689658164978027, -1.390914797782898, 1.4488133192062378, 1.6036533117294312, -3.2666208744049072, -0.8986389636993408, 1.6186678409576416, 2.7323336601257324, 1.1043270826339722, -0.14380814135074615, -0.20710821449756622, -0.5503396987915039, -1.780532717704773, -0.3696193993091583, -0.6430283188819885, 2.59502911567688, -1.8312252759933472, 1.138391137123108, -2.1417367458343506, -0.7669089436531067, -2.1302688121795654, -0.21311882138252258, 0.9402080774307251, 3.4527573585510254, -0.5885308980941772, -2.196575880050659, -1.841490387916565, -4.202951431274414, 0.673316478729248, -2.727597713470459, 1.3007142543792725, -0.5217064619064331, -0.6111270785331726, 0.4391802251338959, 4.620809078216553, -1.233064889907837, 0.7709695100784302, -1.0725597143173218, 2.539665699005127, 2.772324800491333, 0.8293242454528809, 0.3804837763309479, 3.757667064666748, -0.9716252088546753, 3.5218982696533203, 0.05658932775259018, 1.846189260482788, -0.7575887441635132, 2.0531914234161377, -0.8

In [11]:
# len(query_embedding)
len(pass_embedding[1])

4096

## LLAMA 3.2 1B model embeddings

In [6]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3.2:1B",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print(pass_embedding)

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(query_embedding)

Generating embeddings: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]

[[-3.5362844467163086, 4.602611064910889, 2.0321123600006104, -0.3895717263221741, 2.3913087844848633, -3.654560089111328, 0.8637663125991821, 2.4783458709716797, -1.9543064832687378, 0.3272913694381714, -0.4062797725200653, -1.8406726121902466, -4.216486930847168, -0.6841950416564941, -0.20018020272254944, -2.655174732208252, -0.4778594374656677, 0.4474392831325531, 6.593127250671387, -2.2685739994049072, -0.8344859480857849, 1.7138911485671997, -1.1775647401809692, 1.7677785158157349, -1.9011238813400269, -1.3193162679672241, -4.947683334350586, 0.27464792132377625, -1.1102473735809326, 4.34269380569458, -0.5667996406555176, 1.212598443031311, -0.21227000653743744, 0.9419829845428467, -2.819061517715454, -1.056901454925537, 3.3300180435180664, 0.330787718296051, 0.861997663974762, -3.320289373397827, -1.0887861251831055, 0.2743546962738037, 0.9873507618904114, 1.1544866561889648, -0.7765488028526306, -2.27005672454834, -1.163027048110962, -1.7534016370773315, 3.0515189170837402, 1.21

### Multi table query

In [1]:
from llama_index.core import SQLDatabase, ServiceContext

from llama_index.core.query_engine import NLSQLTableQueryEngine

from llama_index.core import (
    Settings,
    VectorStoreIndex,
    StorageContext,
    set_global_handler,
    load_index_from_storage
)

from sqlalchemy import create_engine

from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1:latest", request_timeout=120.0)

# json_mode=True


# Step 1: Set up SQLite connection
engine = create_engine("sqlite:///Databases/AbcInc.db")

# Step 2: Create SQLDatabase instance and include multiple tables
sql_database = SQLDatabase(engine, include_tables=["master", "staging"])

prompt_template = """
You are an expert SQL database designer and developer. 
use `[]` to enclose table and column names.
Do not include any explanations or prose.
"""

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["master", "staging"], 
    llm=llm, handle_sql_errors=True, 
    sql_only=True,
    verbose=True,

    )






In [4]:

query_str = "What are the products mostly shipped to canada"
response = query_engine.query(query_str)
print(response)

Based on the query results, I can synthesize a response as follows:

**Most popular product shipped to Canada:**

The top-selling product in Canada is the **"Samsung Galaxy S22 Ultra"**, with an impressive sales figure of **10,200 units sold**.

This information comes from our recent sales data analysis, which indicates that this particular smartphone model has been flying off the shelves in Canadian stores.


In [67]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# manually set context text
master_text = (
    "This table gives information regarding sales before 31-12-2015"
    " Have the historical data"
)
staging_text = (
    "This table gives information sales after 31-12-2015"
    " Have most recent data"
)

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
# Add SQLTableSchema for both 'master' and 'staging' tables
table_schema_objs = [
    SQLTableSchema(table_name="master", context_str=master_text),
    SQLTableSchema(table_name="staging", context_str=staging_text)  # Add the 'staging' table schema here
] 

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=5), context_str_prefix=db_syntax_rules,
    sql_only=True
)




In [69]:
response = query_engine.query("What are the products mostly shipped to canada?")
print(response)

Based on the query results, the products mostly shipped to Canada are [Item Type 1], [Item Type 2], and [Item Type 3].


In [48]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["master", "staging"], return_raw=True, verbose=True,
    # optional: specify a custom prompt template
    custom_prompt_template=db_syntax_rules
)

In [49]:
results = nl_sql_retriever.retrieve(
    "Which country have highest sales"
)


In [25]:
! pip install matplotlib -q

In [50]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 60bfccb3-980d-4265-9506-046f3a5990cb<br>**Similarity:** None<br>**Text:** Error: Statement 'SELECT Country, SUM(Total Revenue) AS TotalSales\nFROM master\nGROUP BY Country...<br>

In [51]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)
response = query_engine.query(
    "Which country have highest sales"
)
print(str(response))

The country with the highest sales is the one that should be returned by the query.
